# Analysing Fit

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from data_load import load_data, pmumu, nll, lam_i, parabolic, nll_dm2, nll_theta

data, flux = load_data()

In [36]:
# rerunning parabolic minimum calcs, I will use the values given by the parabolic minimiser. 
dm2_0 = 1.5e-3
dm2_1 = 2.4e-3
dm2_2 = 3.5e-3
dm2_min, nll_dm2_min = parabolic(nll_dm2, dm2_0, dm2_1, dm2_2, tol = 1e-5)

print(dm2_min, nll_dm2_min)

t0 = 0.4
t1 = 0.7
t2 = 1.0

theta_min, nll_theta_min = parabolic(nll_theta, t0, t1, t2, tol = 1e-3)

print(f'{theta_min:.3f},{nll_theta_min:.3f}')

0.0024484680873724184 -74.28768561884664
0.674,-73.641


In [45]:


# estimating error by using curvature around minimum + using central finite difference for second deriv
def curv_method(f, min, step_size = 1e-3):
    """ 
    Calculating second derivative to then calculate sigma of the given function

    f = NLL function for 1 variable
    min = minimum value of parameter for given function

    Returns sigma
    """

    f0 = f(min - step_size)
    f1 = f(min)
    f2 = f(min + step_size)

    deriv2 = (f2 - 2 * f1 + f0)/step_size**2
    return np.sqrt(2/deriv2)

sigma_theta = curv_method(nll_theta, theta_min)
sigma_dm2 = curv_method(nll_dm2, dm2_min, step_size = 1e-5)

print(f'sigma theta: {sigma_theta:.3f}, sigma dm2: {sigma_dm2:.6f}')

print(f'Relative Error Theta: {sigma_theta/theta_min * 100:.3f}%')
print(f'Relative Error Dm2: {sigma_dm2/dm2_min * 100:.3f}%')

sigma theta: 0.013, sigma dm2: 0.000011
Relative Error Theta: 1.896%
Relative Error Dm2: 0.449%


In [19]:
def deltaNLL(f, range_min, range_max, points=1000):
    """
    f = NLL
    range_min/max = scan range
    points =  number of grid points

    returns minimum parameter, alongside values where deltaNLL = +-1, and the associated errors
    """

    # range for scanning
    param_range = np.linspace(range_min, range_max, points)
    param_vals = np.array([f(x) for x in param_range])
    # minimum
    index_min = np.argmin(param_vals)
    param_min = param_range[index_min]
    nll_min  = param_vals[index_min]
    target = nll_min + 1.0  

    # upward scan
    i = index_min
    while i < points and param_vals[i] < target:
        i += 1
    if i == points:
        x_plus = param_range[-1]
    else:
        x1, y1 = param_range[i-1], param_vals[i-1]
        x2, y2 = param_range[i],   param_vals[i]
        x_plus = x1 + (target - y1) / (y2 - y1) * (x2 - x1)
    print(f'NLL upwards scan value:{param_vals[i]:.2f}')
    # downward scan
    i = index_min
    while i >= 0 and param_vals[i] < target:
        i -= 1

    if i < 0:
        x_minus = param_range[0]
    else:
        x1, y1 = param_range[i+1], param_vals[i+1]
        x2, y2 = param_range[i],   param_vals[i]
        x_minus = x1 + (target - y1) / (y2 - y1) * (x2 - x1)

    print(f'NLL downwards scan value:{param_vals[i]:.2f}')
    err_plus  = x_plus  - param_min
    err_minus = param_min  - x_minus

    return param_min, x_minus, x_plus, err_minus, err_plus

In [20]:
theta_best, theta_minus, theta_plus, dtheta_minus, dtheta_plus = deltaNLL(nll_theta, range_min = 0.0,range_max = 0.5*np.pi)

print(f'base NLL min val {nll_theta_min:.3f}')

print(f'Theta minimum: {theta_best:.3f}, theta value downwards scan: {theta_minus:.3f}, upwards scan: {theta_plus:.3f}')
print(f'change down: {dtheta_minus:.3f}, change up: {dtheta_plus:.3f}')

NLL upwards scan value:-72.62
NLL downwards scan value:-72.51
base NLL min val -73.641
Theta minimum: 0.676, theta value downwards scan: 0.663, upwards scan: 0.688
change down: 0.013, change up: 0.012


In [50]:
dm2_best, dm2_minus, dm2_plus, ddm2_minus, ddm2_plus = deltaNLL(nll_dm2, range_min = 1.0e-3,range_max = 5.0e-3)

print(f'base NLL min val {nll_dm2_min:.3f}')

print(f'dm2 minimum: {dm2_best:.3f}, dm2 value downwards scan: {dm2_minus:.5f}, upwards scan: {dm2_plus:.5f}')
print(f'change down: {ddm2_minus:.6f}, change up: {ddm2_plus:.6f}')

NLL upwards scan value:-84.83
NLL downwards scan value:-85.07
base NLL min val -74.288
dm2 minimum: 0.002, dm2 value downwards scan: 0.00248, upwards scan: 0.00251
change down: 0.000015, change up: 0.000017


In [48]:
print(f'Error Width theta: {(dtheta_minus + dtheta_plus):.4f}')
print(f'Error Width dm2: {(ddm2_minus +ddm2_plus):.7f}')

print(f'Average Error theta: {(dtheta_minus - dtheta_plus)/2:.4f}')
print(f'Average Error dm2: {(ddm2_minus - ddm2_plus)/2:.7f}')

print(f'Relative Error Theta Minus: {dtheta_minus/theta_best * 100:.3f}%')
print(f'Relative Error Dm2 Minus: {ddm2_minus/dm2_best * 100:.3f}%')

print(f'Relative Error Theta Plus: {dtheta_plus/theta_best * 100:.3f}%')
print(f'Relative Error Dm2 Plus: {ddm2_plus/dm2_best * 100:.3f}%')

Error Width theta: 0.0255
Error Width dm2: 0.0000316
Average Error theta: 0.0004
Average Error dm2: -0.0000011
Relative Error Theta Minus: 1.952%
Relative Error Dm2 Minus: 0.589%
Relative Error Theta Plus: 1.825%
Relative Error Dm2 Plus: 0.676%


# Comparison

# Curvature/Parabolic Method

For the curvature method it was sooooo easy to implement compared to the $\Delta NLL$ method.

Primarily what we do there is:

$$
NLL(x) \approx NLL_{min} + \frac{1}{2}(x-x_{min})^2NLL_{min}
$$

We estimate the value of the NLL at the minimum, calculate the second derivative and then use 

$$
\sigma = \sqrt{\frac{2}{NLL_{min}}}
$$

It produces a single symmetric error and is good to use if the NLL is quadratic as then it agrees well with the $\Delta NLL = 1$. I think we could also use it for error estimation via matrices for multivariable fitting.

However (and very unfortunately) our NLL is not a perfect quadratic and has visible asymmetry (seen in earlier task). This method assumes local quadratic behaviour so it's only an approximation, and also won't give us the true asymmetric errors that we should see.

Another issue is that it's step size dependent. Since we know that our parameters are to 1 d.p or 3 d.p ($\theta$ and $\Delta m^2$ respectively) then we can adjust our step size for each error approximation, but if the expected value is unknown or the step size is otherwise chosen badly, this means that it give out a very incorrect value

# $\Delta NLL = 1$ Method

This method drove me crazy, it was finnicky to code - did not have a good time. But it just simply outclasses the curvature approximation.

It uses the full shape of the NLL and gives asymmetric errors. It also naturally responds well to boundaries as it can still see the correct allowed range. 

The method used is simply to allow a range of values, plug in the nll, and then iterate through the code until $\Delta NLL = 1$. Writing it out now it makes me feel silly that it took me so long... Regardless! Due to various maths that will probably bore you, the forumlation of the NLL, 2lnX, means that upwards and downward scan values for $\Delta NLL = 1$ return $1\sigma$.

This code is taxing computationally, so for larger data sets it could quickly become an issue. The scan range must also be sensible so that $\Delta NLL = 1$ can be reached.

If the NLL is close enough to parabolic near the minimum, there are enough data points and the best/minimum value is inside the allowed range then both methods will work well

# Brief Results Discussion

curvature:
sigma theta: 0.012768479946578124, sigma dm2: 1.0784967348866248e-05

NLL:
average sigma value theta: 0.01276994380007751
average sigma value dm2: 1.5791473522138626e-05

In terms of the returned values, they essentially agree which does suggest that overall the curvature is a good apporximation but we can also see that the dm2 values differ by 5e-6. This isn't terrible, but it highlights were the curvature approximation can fail when only considering the local minimum, as opposed to the whole curve. 

The parabolic minimiser may have guessed incorrectly as to where the minimum NLL for $\Delta m^2$ was as the $\Delta NLL$ code is stating it's actually lower. I'm more inclined to believe the global $\Delta NLL$ scan than the parabolic approximation using local minima.